In [12]:
!tree -l /hps_demo

/hps_demo
└── embedding
    └── hps_infer
        └── 1

3 directories, 0 files


In [14]:
!cp -r ./hps_train0_sparse_1000.model /hps_demo/embedding/hps_infer/1
!cp -r ./hps_train1_sparse_1000.model /hps_demo/embedding/hps_infer/1
!tree -l /hps_demo

/hps_demo
└── embedding
    └── hps_infer
        └── 1
            ├── hps_train0_sparse_1000.model
            │   ├── emb_vector
            │   └── key
            └── hps_train1_sparse_1000.model
                ├── emb_vector
                └── key

5 directories, 4 files


In [9]:
%%writefile /hps_demo/embedding/hps_infer/config.pbtxt
name: "hps_demo"
backend: "hps"
max_batch_size:1024,
input [
  {
    name: "KEYS"
    data_type: TYPE_INT64
    dims: [ -1 ]
  },
  {
    name: "NUMKEYS"
    data_type: TYPE_INT32
    dims: [ -1 ]
  }
]
output [
  {
    name: "OUTPUT0"
    data_type: TYPE_FP32
    dims: [ -1 ]
  }
]
version_policy: {
        specific:{versions: 1}
},
instance_group [
  {
    count: 1
    kind : KIND_GPU
    gpus:[0]
  }
]

Writing ./hps_demo/embedding/hps_infer/config.pbtxt


In [10]:
%%writefile /hps_demo/embedding/hps.json
{
    "supportlonglong": true,
    "volatile_db": {
        "type": "hash_map",
        "user_name": "default",
        "num_partitions": 8,
        "max_get_batch_size": 100000,
        "max_set_batch_size": 100000,
        "overflow_policy": "evict_oldest",
        "overflow_margin": 10000000,
        "overflow_resolution_target": 0.8,
        "initial_cache_rate": 1.0
    },
    "persistent_db": {
        "type": "disabled"
    },
    "models": [{
        "model": "hps_train",
        "sparse_files": ["/hps_demo/embedding/hps_infer/1/hps_train0_sparse_1000.model", "/hps_demo/embedding/hps_infer/1/hps_train1_sparse_1000.model"],
        "num_of_worker_buffer_in_pool": 3,
        "embedding_table_names":["embedding_table1","embedding_table2"],
        "embedding_vecsize_per_table":[1,16],
        "maxnum_catfeature_query_per_table_per_sample":[2,26],
        "default_value_for_each_table":[0.0,0.0],
        "deployed_device_list":[0],
        "max_batch_size":1024,
        "cache_refresh_percentage_per_iteration":0.2,
        "hit_rate_threshold":0.9,
        "gpucacheper":0.5,
        "gpucache":true
        }
    ]
}
# model.fit(max_iter = 1100, display = 200, eval_interval = 1000, snapshot = 1000, snapshot_prefix = "hps_train")
# solver = hugectr.CreateSolver(model_name = "hps_train",
#                               max_eval_batches = 1,
#                               batchsize_eval = 1024,
#                               batchsize = 1024,
#                               lr = 0.001,
#                               vvgpu = [[0]],
#                               i64_input_key = True,
#                               repeat_dataset = True,
#                               use_cuda_graph = True)

Writing /hps_demo/embedding/hps.json


In [11]:
!tree -l ./hps_demo

./hps_demo
└── embedding
    └── hps_infer
        ├── 1
        │   ├── hps_train0_sparse_1000.model
        │   │   ├── emb_vector
        │   │   └── key
        │   └── hps_train1_sparse_1000.model
        │       ├── emb_vector
        │       └── key
        └── config.pbtxt

5 directories, 5 files


In [52]:
!tree /hps_infer/embedding

/hps_infer/embedding
├── hps.json
├── hps_ensemble
│   └── 1
├── hps_torch
│   └── 1
└── hps_wdl
    ├── 1
    │   ├── wdl0_sparse_20000.model
    │   │   ├── emb_vector
    │   │   ├── key
    │   │   ├── wdl0_sparse_20000.model.key
    │   │   └── wdl0_sparse_20000.model.vec
    │   └── wdl1_sparse_20000.model
    │       ├── emb_vector
    │       ├── key
    │       ├── wdl1_sparse_20000.model.key
    │       └── wdl1_sparse_20000.model.vec
    └── config.pbtxt

8 directories, 10 files


In [ ]:
# !tritonserver --model-repository=/wdl_infer/model/ --load-model=wdl \
#     --model-control-mode=explicit \ 
#     --backend-directory=/usr/local/hugectr/backends \
#     --backend-config=hugectr,ps=/wdl_infer/model/ps.json

In [39]:
!tritonserver --model-repository=/hps_infer/embedding/ --load-model=hps_wdl \
    --model-control-mode=explicit \
    --backend-directory=/usr/local/hugectr/backends \
    --backend-config=hps,ps=/hps_infer/embedding/hps.json

I0616 04:50:20.847500 14892 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7fdc46000000' with size 268435456
I0616 04:50:20.903912 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0616 04:50:20.903923 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 1 with size 67108864
I0616 04:50:20.903932 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 2 with size 67108864
I0616 04:50:20.903938 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 3 with size 67108864
I0616 04:50:20.903945 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 4 with size 67108864
I0616 04:50:20.903949 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 5 with size 67108864
I0616 04:50:20.903955 14892 cuda_memory_manager.cc:105] CUDA memory pool is created on device 6 with size 67108864
I0616 04:50:20.903959 14892 cuda_memory_manager.cc:105] CUDA memory

In [41]:
!curl -v localhost:8000/v2/health/ready

*   Trying 127.0.0.1:8000...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8000 (#0)
> GET /v2/health/ready HTTP/1.1
> Host: localhost:8000
> User-Agent: curl/7.68.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Content-Length: 0
< Content-Type: text/plain
< 
* Connection #0 to host localhost left intact


In [47]:
!ls -l /wdl_train/val

total 232
-rw-r--r-- 1 root root  25720 Jun 16 05:07 test.parquet
-rw-r--r-- 1 root root 176913 Jun 16 05:07 train.parquet
-rw-r--r-- 1 root root  25007 Jun 16 05:06 valid.parquet


In [49]:
import pandas as pd
df = pd.read_parquet("/wdl_train/val/test.parquet")
df.head()

,item_id-count,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
3,19,59,"[10, 16, 2, 9, 7, 2, 5, 2, 3, 20, 12, 5, 7, 8,...","[42, 73, 7, 31, 29, 4, 18, 6, 9, 93, 45, 15, 2...","[0.09185880216576592, 0.8937236602103082, 0.54...","[0.5701232580182504, 0.19997826182734457, 0.24..."
14,17,136,"[2, 10, 6, 2, 3, 9, 2, 2, 5, 10, 17, 3, 2, 9, ...","[6, 43, 22, 4, 14, 36, 7, 4, 17, 40, 75, 9, 4,...","[0.6626862922002159, 0.6376980627004402, 0.511...","[0.6466013947223641, 0.015085043467372605, 0.7..."
21,17,174,"[6, 26, 2, 3, 6, 3, 15, 9, 2, 7, 10, 6, 5, 3, ...","[20, 107, 2, 8, 20, 14, 61, 31, 2, 28, 40, 22,...","[0.9958805890923509, 0.5608152309433158, 0.931...","[0.9306965631914249, 0.17375987258739167, 0.07..."
30,17,241,"[9, 9, 4, 25, 6, 5, 5, 3, 5, 2, 2, 8, 4, 9, 9,...","[31, 31, 13, 132, 22, 15, 18, 14, 18, 2, 3, 39...","[0.31851696556214915, 0.20042394643567374, 0.3...","[0.0453928651276444, 0.5975708077512965, 0.816..."
39,16,335,"[2, 9, 11, 20, 10, 11, 14, 9, 10, 3, 4, 4, 44,...","[7, 32, 50, 83, 41, 52, 63, 32, 40, 8, 13, 21,...","[0.6629638255783589, 0.8477154189923084, 0.421...","[0.5507906588971854, 0.4802218018859009, 0.580..."
